# NGSolve in Jupyter-Notebook

In a Jupyter-Book we can intregrate 

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

In [4]:
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…